# **Preprocesamiento y modelado de datos de viviendas**

## **Importación de librerías y carga de datos**

In [ ]:
import pandas as pd
import numpy as np
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Ridge 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor
from sklearn.base import clone
import optuna
import pickle
import bz2
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")

## **Carga del dataset original (2011–2024)**

In [ ]:
df_inicial = pd.read_csv("../data/final/viviendas_2011_2024.csv")
df_inicial.columns

## **Preprocesamiento**

En esta sección se realiza una **limpieza inicial del dataset** para conservar únicamente las variables útiles y mejorar la calidad de los datos:

- **Filtrado temporal:** se conservan solo los registros desde el año **2015**.
- **Eliminación de columnas con exceso de nulos:** se eliminan `Ano_construccion` y `Ano_reforma`.
- **Creación de un indicador de nulos:** se añade `Planta_is_missing` para identificar los casos con valores faltantes en `Planta`.
- **Imputación de valores faltantes:** se imputan los valores nulos de `Planta` con la **mediana por distrito**.
- **Eliminación de variables altamente correlacionadas:** se retiran variables redundantes que no serán utilizadas en el modelado.
- **Verificación final:** se muestran las columnas restantes tras la limpieza.


NOTA: Revisar notebook [EDA_viviendas_2011_2024](./EDA_viviendas_2011_2024.ipynb) para visualizar el análisis que da lugar a las decisiones tomadas en el preprocesamiento.

In [ ]:
df = df_inicial.copy()

df = df[df['Ano'] >= 2015]

# Elminación de Ano_construccion y Ano_reforma por exceso de nulos
df = df.drop(columns=['Ano_construccion', 'Ano_reforma'])

# Crear columna de indicador de nulos para Planta
df['Planta_is_missing'] = df['Planta'].isna().astype(int)

# Imputar valores nulos con la mediana para Planta (por distrito)
df['Planta'] = df.groupby('Distrito')['Planta'].transform(lambda x: x.fillna(x.median()))

# Variables muy correlacionadas y no usadas
var_corr = ['Tamano_vivienda_personas', 'Densidad_poblacion', 'Edad_media', 'Renta_neta_persona',
            'Renta_neta_hogar', 'Renta_bruta_persona', 'Renta_bruta_hogar', 'Precio_predicho']

df = df.drop(columns=var_corr)

print('Columnas restantes:', df.columns.to_list())

En esta etapa se calcula la **tendencia temporal (pendiente lineal)** de diversas variables socioeconómicas y demográficas a lo largo de los años, para cada distrito.

**Proceso:**
- Se define la función `calcular_tendencia`, que ajusta un **modelo de regresión lineal simple** (`LinearRegression`) entre el año (`Ano`) y cada variable de interés.
- Para cada variable:
  - Si los valores son **constantes** a lo largo del tiempo, la tendencia se define como `0.0`.
  - Si existen suficientes datos y variabilidad en `Ano`, se calcula la **pendiente** del modelo (`coef_[0]`), que representa la tasa de cambio anual.
  - Si no hay suficientes datos, se asigna `NaN`.

**Variables consideradas:**
- `Esperanza_vida`  
- `Mayores_65anos%`  
- `Menores_18anos%`  
- `Paro_registrado%`  
- `Apartamentos_turisticos`  
- `Superficie_distrito_ha`  
- `Zonas_verdes%`

El resultado final es un **DataFrame de tendencias por distrito**, donde cada columna indica la **variación anual promedio** de la variable correspondiente.

In [ ]:
from sklearn.linear_model import LinearRegression

def calcular_tendencia(grupo, cols):
    X = grupo['Ano'].values.reshape(-1, 1)
    tendencias = {}
    for c in cols:
        y = grupo[c].values
        if np.allclose(y, y[0], atol=1e-8):  # valores constantes
            tendencias[c] = 0.0
        elif len(np.unique(X)) > 1:
            modelo = LinearRegression().fit(X, y)
            tendencias[c] = modelo.coef_[0]
        else:
            tendencias[c] = np.nan
    return pd.Series(tendencias)

variables = ['Esperanza_vida', 'Mayores_65anos%',
       'Menores_18anos%', 'Paro_registrado%', 'Apartamentos_turisticos',
       'Superficie_distrito_ha', 'Zonas_verdes%']
cols = ['Ano'] + variables
tendencias = df.dropna().groupby('Distrito')[cols].apply(calcular_tendencia, cols=variables).reset_index()

In [ ]:
tendencias

Guardado de tendencias

In [ ]:
tendencias.to_csv("../data/final/tendencias.csv")

## **Imputación de valores faltantes mediante tendencias por distrito**

En esta sección se utilizan las **tendencias lineales calculadas previamente** para imputar los valores faltantes de ciertas variables socioeconómicas a nivel de distrito.

**Resultado:**  
Un dataset sin valores nulos en las variables seleccionadas, imputadas de forma coherente con la evolución temporal observada en cada distrito.


In [ ]:
# Renombrar columnas de tendencias para evitar confusión
tendencias_rename = {var: f'{var}_tendencia' for var in variables}
tendencias = tendencias.rename(columns=tendencias_rename)

# Aplicar por distrito (agregado include_groups=False)
vars_faltantes = ['Mayores_65anos%', 'Menores_18anos%', 'Paro_registrado%']

# Seleccionar solo columna Distrito y las tendencias que necesitamos
tendencias_merge = tendencias[['Distrito'] + [f'{var}_tendencia' for var in vars_faltantes]]

# Merge limpio
df = df.merge(tendencias_merge, on='Distrito', how='left')

# Crear indicadores de nulos
for var in vars_faltantes:
    df[f"{var}_is_missing"] = df[var].isna().astype(int)

# Función de imputación con tendencia
def imputar_con_tendencia(grupo, var):

    # Pendiente anual (tasa de cambio por año) a usar para extrapolación futura.
    tendencia = grupo[f'{var}_tendencia'].iloc[0]
    ultimo_valido = grupo[var].last_valid_index()
    
    # Si existe al menos un dato observado, extrapola SOLO para años posteriores.
    if ultimo_valido is not None:
        
        # Último valor y su año de referencia (punto base para la extrapolación lineal).
        ultimo_valor = grupo.loc[ultimo_valido, var]
        ultimo_ano = grupo.loc[ultimo_valido, 'Ano']
        
        # Imputa valores nulos en años futuros (> último_ano) extrapolando linealmente desde el último valor conocido usando la tendencia.
        mask_futuro = grupo[var].isna() & (grupo['Ano'] > ultimo_ano)
        grupo.loc[mask_futuro, var] = (ultimo_valor + tendencia * (grupo.loc[mask_futuro, 'Ano'] - ultimo_ano))
    
    # Relleno final de nulos restantes: ffill/bfill para nulos restantes
    return grupo[var].ffill().bfill()

for var in vars_faltantes:
    df[var] = df.groupby('Distrito', group_keys=False).apply(lambda g: imputar_con_tendencia(g, var), include_groups=False)

En esta última etapa se realiza una **limpieza posterior a la imputación** y una verificación general del estado del dataset.

**Eliminación de columnas auxiliares de tendencia:**  
   Las columnas utilizadas para la imputación (`*_tendencia`) ya no son necesarias,  
   por lo que se eliminan del DataFrame.

In [ ]:
# Eliminar columnas de tendencias que ya no son necesarias tras la imputación
cols_tendencia = [col for col in df.columns if '_tendencia' in col]
df = df.drop(columns=cols_tendencia)

# Verificar estado final del dataset: tipos de datos y ausencia de nulos
resumen = pd.DataFrame({"Tipo": df.dtypes, "Nulos": df.isna().sum(), "Nulos_%": (df.isna().mean()*100).round(2)})
print(resumen)

## **Creación de variables temporales y de contexto de mercado**

En esta sección se generan **nuevas características (features)** que capturan la **dinámica temporal y espacial del mercado inmobiliario**, con el fin de enriquecer el modelo predictivo.

**Resultado:**  
Se añaden múltiples **features temporales y de contexto local** que capturan la evolución de los precios, la estabilidad del mercado y la posición relativa de cada vivienda dentro de su entorno.

In [ ]:
# Ordenar el dataframe por las columnas clave para calcular correctamente los lags temporales
df = df.sort_values(['Distrito', 'Tipo_vivienda', 'Ano']).reset_index(drop=True)

# --- FEATURE 1: Precio promedio año anterior por Distrito + Tipo_vivienda ---
# Compara cada vivienda con el precio medio de viviendas similares del año anterior
df['precio_prom_ano_ant'] = (df.groupby(['Distrito', 'Tipo_vivienda'])['Precio_ajustado'].shift(1))

# --- FEATURE 2: Tendencia de precio por Distrito (cambio porcentual) ---
# Captura la dinámica del mercado en cada distrito
df['tendencia_distrito'] = (
    df.groupby('Distrito')['Precio_ajustado']
    .pct_change()  # Variación porcentual respecto al periodo anterior
)

# --- FEATURE 3: Precio por m² del año anterior por Distrito ---
# Normaliza por tamaño para comparar valor relativo
df['precio_m2'] = df['Precio_ajustado'] / df['Tamano']
df['precio_m2_distrito_ant'] = (
    df.groupby('Distrito')['precio_m2']
    .shift(1)
)

# Eliminar columna auxiliar
df = df.drop(columns=['precio_m2'])

# --- FEATURE 4 (OPCIONAL): Ratio precio vs promedio del distrito ---
# Indica si la vivienda está por encima/debajo del mercado local
df['precio_prom_distrito'] = df.groupby(['Distrito', 'Ano'])['Precio_ajustado'].transform('mean')
df['ratio_vs_distrito'] = df['Precio_ajustado'] / df['precio_prom_distrito']
df = df.drop(columns=['precio_prom_distrito'])

# --- FEATURE 5 (OPCIONAL): Volatilidad del distrito ---
# Mide la variabilidad de precios en cada zona
df['volatilidad_distrito'] = (
    df.groupby(['Distrito', 'Ano'])['Precio_ajustado']
    .transform('std')  # Desviación estándar como proxy de volatilidad
)

# Imputar nulos resultantes de los lags (primer año de cada grupo)
# Usamos mediana por distrito para mantener coherencia espacial
for col in ['precio_prom_ano_ant', 'tendencia_distrito', 'precio_m2_distrito_ant', 'volatilidad_distrito']:
    if col in df.columns:
        df[col] = df.groupby('Distrito')[col].transform(lambda x: x.fillna(x.median()))

print("Features temporales agregadas creadas correctamente")
print(f"Nuevas columnas: {[c for c in df.columns if 'prom_ano' in c or 'tendencia' in c or 'm2_distrito' in c or 'ratio_vs' in c or 'volatilidad' in c]}")

## **Preparación del dataset y configuración del entrenamiento**

En esta sección se preparan los datos y la configuración necesaria para entrenar y evaluar distintos modelos predictivos.  
El objetivo es construir un flujo de trabajo **reproducible, limpio y compatible con validación temporal.**

**Resultado:**  
Dataset y entorno de entrenamiento completamente configurados, con los modelos listos para iniciar la fase de validación y comparación de desempeño.

In [ ]:
# --- 1. Encoding de variables categóricas ---
df = pd.get_dummies(df, drop_first=True)
print(f"Shape después de get_dummies: {df.shape}")

# --- 2. Preparar dataset ---
df = df.sort_values(by="Ano").reset_index(drop=True)
df = df.dropna(subset=["Precio_ajustado"])
print(f"Dataset ordenado y limpio: {df.shape}")

# --- 3. Definir X / y ---
TARGET_COL = "Precio_ajustado"
DROP_COLS = ["Ano", TARGET_COL]

FEATURES = [c for c in df.columns if c not in DROP_COLS]
print(f"Total de features: {len(FEATURES)}")

# --- 4. Separación temporal y por venta y alquiler---
mask_train_venta = (df["Ano"] <= 2022) & (df['Operacion_venta'] == 1)
mask_train_alquiler = (df["Ano"] <= 2022) & (df['Operacion_venta'] == 0)
mask_test_venta = (df["Ano"] >= 2023) & (df['Operacion_venta'] == 1)
mask_test_alquiler = (df["Ano"] >= 2023) & (df['Operacion_venta'] == 0)

X_train_venta = df.loc[mask_train_venta, FEATURES]
y_train_venta = df.loc[mask_train_venta, TARGET_COL]
X_train_alquiler = df.loc[mask_train_alquiler, FEATURES]
y_train_alquiler = df.loc[mask_train_alquiler, TARGET_COL]

X_test_venta = df.loc[mask_test_venta, FEATURES]
y_test_venta = df.loc[mask_test_venta, TARGET_COL]
X_test_alquiler = df.loc[mask_test_alquiler, FEATURES]
y_test_alquiler = df.loc[mask_test_alquiler, TARGET_COL]

print(f"Train venta: {X_train_venta.shape}, Test venta: {X_test_venta.shape}")
print(f"Train alquiler: {X_train_alquiler.shape}, Test alquiler: {X_test_alquiler.shape}")

# --- 5. Definir pipeline (UNA SOLA VEZ) ---
def definir_pipeline(modelo) -> Pipeline:
    """
    Aplica escalado RobustScaler + modelo.
    
    Parameters
    ----------
    modelo : estimator
        Modelo de sklearn (Ridge, RandomForest, etc.)
    
    Returns
    -------
    pipeline : Pipeline
        Pipeline con scaler + modelo
    """
    pipeline = Pipeline([
        ('scaler', RobustScaler()), 
        ('modelo', modelo)
    ])
    return pipeline

# --- 6. Función de evaluación ---
def evaluar(model, X_tr, y_tr, X_va, y_va):
    """
    Entrena y evalúa un modelo con métricas completas.
    """
    model.fit(X_tr, y_tr)
    pred_tr = model.predict(X_tr)
    pred_va = model.predict(X_va)

    def mape(y_true, y_pred):
        eps = 1e-9
        return np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + eps))) * 100

    return {
        "MAE_train": mean_absolute_error(y_tr, pred_tr),
        "RMSE_train": np.sqrt(mean_squared_error(y_tr, pred_tr)),
        "R2_train": r2_score(y_tr, pred_tr),
        "MAE_val": mean_absolute_error(y_va, pred_va),
        "RMSE_val": np.sqrt(mean_squared_error(y_va, pred_va)),
        "MAPE_val_%": mape(y_va, pred_va),
        "R2_val": r2_score(y_va, pred_va),
    }

# --- 7. Definir modelos ---
modelos = {
    "ridge": definir_pipeline(Ridge(alpha=1.0, random_state=42)),
    "knn":   definir_pipeline(KNeighborsRegressor(n_neighbors=7, weights="distance", n_jobs=-1)),
    "hgb":   definir_pipeline(HistGradientBoostingRegressor(loss="absolute_error",
                max_depth=15, learning_rate=0.06, max_iter=500, random_state=42)),
    "lgbm":  definir_pipeline(LGBMRegressor(n_estimators=1000, learning_rate=0.05,
                subsample=0.8, colsample_bytree=0.8, force_row_wise=True,
                verbosity=-1, n_jobs=-1, random_state=42)),
    "mlp":   definir_pipeline(MLPRegressor(
                hidden_layer_sizes=(64, 32),
                max_iter=800,
                activation="relu",
                learning_rate_init=1e-3,
                early_stopping=True,
                n_iter_no_change=10,
                random_state=42)),
}

# --- 8. Validación temporal con TimeSeriesSplit ---
splits = 5
tscv = TimeSeriesSplit(n_splits=splits)
resultados_venta = {}
entrenados_venta = {}
resultados_alquiler = {}
entrenados_alquiler = {}

print("Setup completado. Listo para entrenar modelos.")
print(df.columns)

## **Entrenamiento y validación temporal de modelos**

En esta sección se realiza el **entrenamiento con validación temporal** utilizando el método `TimeSeriesSplit`, asegurando que la evaluación de los modelos respete la secuencia cronológica de los datos.

**Resultado:**  
Se obtienen modelos validados temporalmente, con métricas de desempeño en distintos periodos,  
y versiones finales entrenadas con todos los datos históricos disponibles.

**Entrenamientos de prueba para VENTA y ALQUILER:**

In [ ]:
print("Iniciando entrenamiento con TimeSeriesSplit...\n")

# Entrenamientos de prueba para VENTA
for nombre, pipe in modelos.items():
    print(f"Entrenando modelo: {nombre.upper()}")
    resultados_modelo = []
    n_validacion = 0
    
    for train_index, val_index in tscv.split(X_train_venta):
        n_validacion += 1
        print(f"   Split {n_validacion}/{splits}")

        pipe_cv = clone(pipe)

        X_tr, X_val = X_train_venta.iloc[train_index], X_train_venta.iloc[val_index]
        y_tr, y_val = y_train_venta.iloc[train_index], y_train_venta.iloc[val_index]
        
        metrics = evaluar(pipe_cv, X_tr, y_tr, X_val, y_val)
        resultados_modelo.append(metrics)

    resultados_venta[nombre] = resultados_modelo

    # Entrenamiento final con todo el train set
    print(f"   Fit final en {X_train_venta.shape[0]} muestras\n")
    pipe_final = clone(pipe).fit(X_train_venta, y_train_venta)
    entrenados_venta[nombre] = pipe_final

In [ ]:
print("Iniciando entrenamiento con TimeSeriesSplit...\n")

# Entrenamientos de prueba para ALQUILER
for nombre, pipe in modelos.items():
    print(f"Entrenando modelo: {nombre.upper()}")
    resultados_modelo = []
    n_validacion = 0
    
    for train_index, val_index in tscv.split(X_train_alquiler):
        n_validacion += 1
        print(f"   Split {n_validacion}/{splits}")

        pipe_cv = clone(pipe)

        X_tr, X_val = X_train_alquiler.iloc[train_index], X_train_alquiler.iloc[val_index]
        y_tr, y_val = y_train_alquiler.iloc[train_index], y_train_alquiler.iloc[val_index]
        
        metrics = evaluar(pipe_cv, X_tr, y_tr, X_val, y_val)
        resultados_modelo.append(metrics)

    resultados_alquiler[nombre] = resultados_modelo

    # Entrenamiento final con todo el train set
    print(f"   Fit final en {X_train_alquiler.shape[0]} muestras\n")
    pipe_final = clone(pipe).fit(X_train_alquiler, y_train_alquiler)
    entrenados_alquiler[nombre] = pipe_final

**Mostrar resultados de validación cruzada**

En esta sección se presentan los **resultados de la validación temporal (2015–2022)** para todos los modelos entrenados.  
El objetivo es comparar su desempeño y seleccionar el modelo más prometedor según las métricas de evaluación.

Un resumen claro del rendimiento de cada modelo, destacando el mejor candidato para predicciones futuras.

In [ ]:
print("\n" + "="*70)
print("RESULTADOS DE VALIDACIÓN CRUZADA DE VENTA (2015-2022)")
print("="*70)

ranking = sorted(
    resultados_venta.items(),
    key=lambda kv: min([m["MAE_val"] for m in kv[1]])
)

for nombre, metricas in ranking:
    mae_vals = [m["MAE_val"] for m in metricas]
    rmse_vals = [m["RMSE_val"] for m in metricas]
    r2_vals = [m["R2_val"] for m in metricas]
    mape_vals = [m["MAPE_val_%"] for m in metricas]

    mae_tr_vals = [m["MAE_train"] for m in metricas]
    rmse_tr_vals = [m["RMSE_train"] for m in metricas]
    r2_tr_vals = [m["R2_train"] for m in metricas]

    print(f"\n{nombre.upper()}:")
    print(f"  Validación:")
    print(f"    MAE    = {min(mae_vals):,.2f} EUR")
    print(f"    RMSE   = {min(rmse_vals):,.2f} EUR")
    print(f"    R2     = {max(r2_vals):.4f}")
    print(f"    MAPE   = {min(mape_vals):.2f}%")
    print(f"  Entrenamiento:")
    print(f"    MAE    = {min(mae_tr_vals):,.2f} EUR")
    print(f"    RMSE   = {min(rmse_tr_vals):,.2f} EUR")
    print(f"    R2     = {max(r2_tr_vals):.4f}")

mejor_nombre_venta = ranking[0][0]
mejor_modelo_venta = entrenados_venta[mejor_nombre_venta]

print("\n" + "="*70)
print(f"MEJOR MODELO: {mejor_nombre_venta.upper()}")
print("="*70)

In [ ]:
print("\n" + "="*70)
print("RESULTADOS DE VALIDACIÓN CRUZADA DE ALQUILER (2015-2022)")
print("="*70)

ranking = sorted(
    resultados_alquiler.items(),
    key=lambda kv: min([m["MAE_val"] for m in kv[1]])
)

for nombre, metricas in ranking:
    mae_vals = [m["MAE_val"] for m in metricas]
    rmse_vals = [m["RMSE_val"] for m in metricas]
    r2_vals = [m["R2_val"] for m in metricas]
    mape_vals = [m["MAPE_val_%"] for m in metricas]

    mae_tr_vals = [m["MAE_train"] for m in metricas]
    rmse_tr_vals = [m["RMSE_train"] for m in metricas]
    r2_tr_vals = [m["R2_train"] for m in metricas]

    print(f"\n{nombre.upper()}:")
    print(f"  Validación:")
    print(f"    MAE    = {min(mae_vals):,.2f} EUR")
    print(f"    RMSE   = {min(rmse_vals):,.2f} EUR")
    print(f"    R2     = {max(r2_vals):.4f}")
    print(f"    MAPE   = {min(mape_vals):.2f}%")
    print(f"  Entrenamiento:")
    print(f"    MAE    = {min(mae_tr_vals):,.2f} EUR")
    print(f"    RMSE   = {min(rmse_tr_vals):,.2f} EUR")
    print(f"    R2     = {max(r2_tr_vals):.4f}")

mejor_nombre_alquiler = ranking[0][0]
mejor_modelo_alquiler = entrenados_alquiler[mejor_nombre_alquiler]

print("\n" + "="*70)
print(f"MEJOR MODELO: {mejor_nombre_alquiler.upper()}")
print("="*70)

## **Optimización de hiperparámetros con Optuna (Random Forest)**

En esta sección se realiza la **optimización automática de hiperparámetros** del mejor modelo utilizando la librería **Optuna**, con el objetivo de minimizar el error medio absoluto (MAE) en validación temporal.

**Resultado:**  
Un modelo optimizado mediante búsqueda automática de hiperparámetros,  
validado temporalmente y listo para compararse con los demás modelos base.

In [ ]:
def objective_lgbm_venta(trial):
    """
    Función objetivo para Optuna: minimiza MAE promedio en validación cruzada.
    """
    params = {
        "objective": "regression",
        "metric": "mae",
        "verbose": -1,
        "boosting_type": "gbdt",
        "n_estimators": trial.suggest_int("n_estimators", 500, 1000, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.05, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 15, 255),
        "max_depth": -1,
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100, step=5),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 0.3),
        "feature_fraction_bynode": trial.suggest_float("feature_fraction_bynode", 0.6, 1.0),
        "random_state": 42,
        "n_jobs": -1,
    }

    pipe = definir_pipeline(LGBMRegressor(**params))

    maes = []
    for train_idx, val_idx in tscv.split(X_train_venta):
        pipe_cv = clone(pipe)

        X_tr, X_val = X_train_venta.iloc[train_idx], X_train_venta.iloc[val_idx]
        y_tr, y_val = y_train_venta.iloc[train_idx], y_train_venta.iloc[val_idx]
        
        mets = evaluar(pipe_cv, X_tr, y_tr, X_val, y_val)
        maes.append(mets["MAE_val"])

    return np.mean(maes)

def objective_lgbm_alquiler(trial):
    """
    Función objetivo para Optuna: minimiza MAE promedio en validación cruzada.
    """
    params = {
        "objective": "regression",
        "metric": "mae",
        "verbose": -1,
        "boosting_type": "gbdt",
        "n_estimators": trial.suggest_int("n_estimators", 500, 1000, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 15, 255),
        "max_depth": -1,
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100, step=5),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 0.3),
        "feature_fraction_bynode": trial.suggest_float("feature_fraction_bynode", 0.6, 1.0),
        "random_state": 42,
        "n_jobs": -1,
    }

    pipe = definir_pipeline(LGBMRegressor(**params))

    maes = []
    for train_idx, val_idx in tscv.split(X_train_alquiler):
        pipe_cv = clone(pipe)

        X_tr, X_val = X_train_alquiler.iloc[train_idx], X_train_alquiler.iloc[val_idx]
        y_tr, y_val = y_train_alquiler.iloc[train_idx], y_train_alquiler.iloc[val_idx]
        
        mets = evaluar(pipe_cv, X_tr, y_tr, X_val, y_val)
        maes.append(mets["MAE_val"])

    return np.mean(maes)

In [ ]:
print("\nIniciando optimización de hiperparámetros para VENTA con Optuna...")
print(f"Usando TimeSeriesSplit con {splits} splits\n")

# Crear estudio de Optuna para VENTA
study_venta = optuna.create_study(direction="minimize", study_name="Optimizacion_LGBM_venta")
study_venta.optimize(objective_lgbm_venta, n_trials=40, show_progress_bar=True)

print("\n" + "="*70)
print("RESULTADOS DE OPTIMIZACIÓN")
print("="*70)
print(f"Mejor MAE validación: {study_venta.best_value:,.2f} EUR")
print(f"\nMejores hiperparámetros:")
for param, value in study_venta.best_params.items():
    print(f"  {param}: {value}")

In [ ]:
print("\nIniciando optimización de hiperparámetros para ALQUILER con Optuna...")
print(f"Usando TimeSeriesSplit con {splits} splits\n")

# Crear estudio de Optuna para ALQUILER
study_alquiler = optuna.create_study(direction="minimize", study_name="Optimizacion_LGBM_alquiler")
study_alquiler.optimize(objective_lgbm_alquiler, n_trials=40, show_progress_bar=True)

print("\n" + "="*70)
print("RESULTADOS DE OPTIMIZACIÓN")
print("="*70)
print(f"Mejor MAE validación: {study_alquiler.best_value:,.2f} EUR")
print(f"\nMejores hiperparámetros:")
for param, value in study_alquiler.best_params.items():
    print(f"  {param}: {value}")

**Entrenamiento del modelo Random Forest final con los mejores hiperparámetros**

Se entrena el modelo Random Forest utilizando los hiperparámetros óptimos encontrados con Optuna. El modelo final se integra en un pipeline con escalado robusto y se ajusta sobre los datos de entrenamiento completos, obteniendo así la versión definitiva lista para evaluación y predicciones futuras.

In [ ]:
# ===== Entrenar el mejor modelo final =====
lgbm_final_venta = definir_pipeline(LGBMRegressor(**study_venta.best_params, n_jobs=-1, random_state=42))
lgbm_final_alquiler = definir_pipeline(LGBMRegressor(**study_alquiler.best_params, random_state=42))

# Entrenar con todos los datos (ya que es una serie temporal)
lgbm_final_venta.fit(X_train_venta, y_train_venta)
lgbm_final_alquiler.fit(X_train_alquiler, y_train_alquiler)

Guardar modelos

In [ ]:
with bz2.open('../models/lgbm_final_venta.pkl.bz2', 'wb') as f:
    pickle.dump(lgbm_final_venta, f)

In [ ]:
with bz2.open('../models/lgbm_final_alquiler.pkl.bz2', 'wb') as f:
    pickle.dump(lgbm_final_alquiler, f)

Cargar modelo

In [ ]:
with bz2.open('../models/lgbm_final_venta.pkl.bz2', 'rb') as f:
    lgbm_final_venta = pickle.load(lgbm_final_venta, f)
with bz2.open('../models/lgbm_final_alquiler.pkl.bz2', 'rb') as f:
    lgbm_final_alquiler = pickle.load(lgbm_final_alquiler, f)

**Análisis de la importancia de variables en el modelo Random Forest**

Se extraen y visualizan las variables con mayor peso en el modelo final de Random Forest. Este análisis permite identificar los factores que más influyen en la predicción del precio ajustado, mostrando las 15 variables más relevantes mediante una tabla y un gráfico de barras.

In [ ]:
import matplotlib.pyplot as plt

# Extraer importancias
importancias = rf_final.named_steps['modelo'].feature_importances_
features = X_train.columns

# Crear DataFrame ordenado
imp_df = pd.DataFrame({
    'Variable': features,
    'Importancia': importancias
}).sort_values(by='Importancia', ascending=False).head(15)

# Mostrar tabla y gráfico
display(imp_df)

plt.figure(figsize=(8,5))
plt.barh(imp_df['Variable'], imp_df['Importancia'])
plt.gca().invert_yaxis()
plt.title('Top 15 variables más importantes en el modelo Random Forest')
plt.xlabel('Importancia')
plt.show()

Evaluación en 2023 y 2024

In [ ]:
# Verificar que X_test no tenga nulos
print("Nulos en X_test:")
print(X_test.isna().sum().sum())

# Si no hay nulos, evaluar directamente
y_pred_test = rf_final.predict(X_test)

print("\n" + "="*50)
print("EVALUACIÓN FINAL EN TEST (2023-2024)")
print("="*50)

print(f"MAE:  {mean_absolute_error(y_test, y_pred_test):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")
print(f"R²:   {r2_score(y_test, y_pred_test):.4f}")

# MAPE
mape_test = np.mean(np.abs((y_test - y_pred_test) / (np.abs(y_test) + 1e-9))) * 100
print(f"MAPE: {mape_test:.2f}%")

# Comparación visual
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test, alpha=0.5, edgecolors='k', linewidth=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Predicción perfecta')
plt.xlabel('Precio Real', fontsize=12)
plt.ylabel('Precio Predicho', fontsize=12)
plt.title('Predicciones vs Realidad (Test 2023-2024)', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Análisis de errores
errores = y_test - y_pred_test
errores_abs = np.abs(errores)
errores_rel = np.abs(errores / y_test) * 100

print("Análisis de errores:")
print(f"Error medio: {errores.mean():.2f} (sesgo)")
print(f"Mediana del error absoluto: {np.median(errores_abs):.2f}")
print(f"Percentil 90 del error relativo: {np.percentile(errores_rel, 90):.2f}%")
print(f"% predicciones con error <10%: {(errores_rel < 10).mean() * 100:.1f}%")
print(f"% predicciones con error >50%: {(errores_rel > 50).mean() * 100:.1f}%")

# Identificar peores predicciones
peores = pd.DataFrame({
    'Real': y_test,
    'Predicho': y_pred_test,
    'Error_abs': errores_abs,
    'Error_rel_%': errores_rel
}).sort_values('Error_abs', ascending=False).head(10)

print("\nPeores 10 predicciones:")
display(peores)

In [ ]:
# Crear rangos de precio
bins = [0, 300000, 600000, 1000000, 2000000, np.inf]
labels = ['<300k', '300-600k', '600k-1M', '1-2M', '>2M']

df_errores = pd.DataFrame({
    'Real': y_test.values,
    'Predicho': y_pred_test,
    'Error_abs': errores_abs,
    'Error_rel_%': errores_rel
})

df_errores['Rango'] = pd.cut(df_errores['Real'], bins=bins, labels=labels)

# Análisis por rango
print("Rendimiento por rango de precio:\n")
resumen_rangos = df_errores.groupby('Rango').agg({
    'Error_abs': ['mean', 'median'],
    'Error_rel_%': ['mean', 'median'],
    'Real': 'count'
}).round(2)

resumen_rangos.columns = ['MAE', 'Mediana_error', 'Error_rel_%_medio', 'Error_rel_%_mediano', 'n_casos']
display(resumen_rangos)

# Visualizar errores por rango
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
df_errores.boxplot(column='Error_rel_%', by='Rango', ax=plt.gca())
plt.title('Distribución del error relativo por rango de precio')
plt.ylabel('Error relativo (%)')
plt.xlabel('Rango de precio')
plt.suptitle('')

plt.subplot(1, 2, 2)
resumen_rangos['Error_rel_%_medio'].plot(kind='bar', color='steelblue')
plt.title('Error relativo promedio por rango')
plt.ylabel('Error relativo medio (%)')
plt.xlabel('Rango de precio')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Identificar casos problemáticos en <300k
df_bajo = df_errores[df_errores['Rango'] == '<300k']
casos_extremos = df_bajo[df_bajo['Error_rel_%'] > 100]

print(f"Casos con error >100% en segmento <300k: {len(casos_extremos)} ({len(casos_extremos)/len(df_bajo)*100:.2f}%)")
print(f"\nEstadísticas de estos casos extremos:")
print(f"Precio real promedio: {casos_extremos['Real'].mean():.2f}")
print(f"Precio predicho promedio: {casos_extremos['Predicho'].mean():.2f}")

# Obtener índices para investigar features
indices_extremos = casos_extremos.index.tolist()[:5]
print(f"\nPrimeros 5 casos extremos (índices): {indices_extremos}")

In [ ]:
def evaluar_sin_fit(model, X_tr, y_tr, X_va, y_va):
    pred_tr = model.predict(X_tr)
    pred_va = model.predict(X_va)

    def mape(y_true, y_pred):
        eps = 1e-9
        return np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + eps))) * 100

    return {
        "MAE_train": mean_absolute_error(y_tr, pred_tr),
        "RMSE_train": np.sqrt(mean_squared_error(y_tr, pred_tr)),
        "R2_train": r2_score(y_tr, pred_tr),
        "MAE_val": mean_absolute_error(y_va, pred_va),
        "RMSE_val": np.sqrt(mean_squared_error(y_va, pred_va)),
        "MAPE_val_%": mape(y_va, pred_va),
        "R2_val": r2_score(y_va, pred_va),
    }

In [ ]:
print("\n--- Evaluación en 2023 ---")
eval_2023 = evaluar_sin_fit(rf_final, X_train, y_train,
                    X_test.loc[df["Ano"] == 2023],
                    y_test.loc[df["Ano"] == 2023])
print(eval_2023)

print("\n--- Evaluación en 2024 ---")
eval_2024 = evaluar_sin_fit(rf_final, X_train, y_train,
                    X_test.loc[df["Ano"] == 2024],
                    y_test.loc[df["Ano"] == 2024])
print(eval_2024)

**Comparativa de desempeño entre el modelo base y el modelo optimizado**

Se comparan los resultados del modelo Random Forest inicial frente al modelo optimizado tras la búsqueda de hiperparámetros. El modelo ajustado muestra una mejora en el MAE y MAPE de validación, manteniendo un R² alto, lo que confirma una mayor precisión y mejor capacidad de generalización.

In [ ]:
rf_base = entrenados["rf"]  # el modelo base antes del tuning

base_mets = resultados["rf"]
tuned_mets = evaluar(rf_final, X_train, y_train, X_val, y_val)

print("\n--- Comparativa Random Forest ---")
print("Base :", base_mets)
print("Tuned:", tuned_mets)

In [ ]:
# Asegurarnos de usar todo el test set
y_pred_test_final = rf_final.predict(X_test)

print("\n" + "="*60)
print("MÉTRICAS FINALES EN TEST COMPLETO (2023-2024)")
print("="*60)
print(f"MAE:  {mean_absolute_error(y_test, y_pred_test_final):,.2f} EUR")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test_final)):,.2f} EUR")
print(f"R²:   {r2_score(y_test, y_pred_test_final):.4f}")

mape_final = np.mean(np.abs((y_test - y_pred_test_final) / (np.abs(y_test) + 1e-9))) * 100
print(f"MAPE: {mape_final:.2f}%")
print("="*60)